In [1]:
import pandas as pd
import numpy as np
from geopy.distance import distance
import pydeck as pdk
import pickle
import requests
import json

In [2]:
model = pickle.load(open('random_forest.pickle', 'rb'))

In [3]:
data = pd.read_csv('../../../capstone_other/files_too_large_for_github/ML_table_with_availability_clusters.csv')

In [4]:
new = data.groupby('dock_id').apply(pd.DataFrame.sample, random_state = 5, n=1).reset_index(drop=True)

In [5]:
new1 = new[['dock_id', 'latitude', 'longitude', 'tot_docks']]

In [6]:
new2 = new[['weekday_cluster', 'weekend_cluster']]

In [7]:
month = 12
day = 18
hour = 13

In [8]:
date = pd.to_datetime({'year': [2021], 'month': [month], 'day': [day], 'hour': [hour]})[0]

In [9]:
day_of_week_conversion = {0:2, 1:3, 2:4, 3:5, 4:6, 5:7, 6:1}

In [10]:
sql_day_of_week = day_of_week_conversion[date.dayofweek]

In [11]:
sql_day_of_week

7

In [12]:
month_column = np.ones(len(new), dtype=int) * month
num_day_column = np.ones(len(new), dtype=int) * sql_day_of_week
hour_column = np.ones(len(new), dtype=int) * hour
date_df = pd.DataFrame({'month': month_column, 'num_day': num_day_column, 'hour': hour_column})

In [13]:
date_df

,month,num_day,hour
0,12,7,13
1,12,7,13
2,12,7,13
3,12,7,13
4,12,7,13
...,...,...,...
324,12,7,13
325,12,7,13
326,12,7,13
327,12,7,13


In [14]:
request = requests.get('https://api.openweathermap.org/data/2.5/onecall?lat=40.7812&lon=-73.9665&exclude=minutely,current,daily,alerts&units=metric&appid=404310456b8e1c31228341dd6c95dd04')
response = request.json()
response

{'lat': 40.7812,
 'lon': -73.9665,
 'timezone': 'America/New_York',
 'timezone_offset': -18000,
 'hourly': [{'dt': 1639789200,
   'temp': 9.47,
   'feels_like': 7.67,
   'pressure': 1022,
   'humidity': 31,
   'dew_point': -6.04,
   'uvi': 0,
   'clouds': 1,
   'visibility': 10000,
   'wind_speed': 3.36,
   'wind_deg': 298,
   'wind_gust': 8.36,
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01n'}],
   'pop': 0},
  {'dt': 1639792800,
   'temp': 9.54,
   'feels_like': 7.85,
   'pressure': 1022,
   'humidity': 34,
   'dew_point': -4.91,
   'uvi': 0,
   'clouds': 20,
   'visibility': 10000,
   'wind_speed': 3.19,
   'wind_deg': 305,
   'wind_gust': 8.17,
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02n'}],
   'pop': 0},
  {'dt': 1639796400,
   'temp': 9.43,
   'feels_like': 7.5,
   'pressure': 1022,
   'humidity': 37,
   'dew_point': -4,
   'uvi': 0,
   'clouds': 39,
   'visibility': 1000

In [15]:
temp = response['hourly'][22]['temp']
rain = round(response['hourly'][22]['pop'])

In [16]:
weather_df = pd.DataFrame.from_dict(response['hourly'])[['dt', 'temp','pop']]
weather_df['dt'] = pd.to_datetime(weather_df['dt'], unit = 's')

In [17]:
weather_df = weather_df[weather_df['dt'] == date]

In [18]:
temp = weather_df['temp'].iloc[0]
rain = round(weather_df['pop'].iloc[0])

In [19]:
temp_column = np.ones(len(new), dtype=int) * temp
rain_column = np.ones(len(new), dtype=int) * rain

In [20]:
new1['tmp'] = temp_column
new1['rain'] = rain_column

<ipython-input-20-6b2e212f92bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new1['tmp'] = temp_column
<ipython-input-20-6b2e212f92bf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new1['rain'] = rain_column


In [21]:
day_data = pd.concat([new1, new2], axis = 1)

In [22]:
day_data

,dock_id,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster
0,83,40.683826,-73.976323,62,6.12,1,2,3
1,119,40.696089,-73.978034,19,6.12,1,2,0
2,120,40.686768,-73.959282,19,6.12,1,1,0
3,143,40.692395,-73.993379,24,6.12,1,2,3
4,144,40.698399,-73.980689,19,6.12,1,2,0
...,...,...,...,...,...,...,...,...
324,3505,40.805726,-73.936322,33,6.12,1,2,2
325,3506,40.801307,-73.939817,29,6.12,1,0,2
326,3507,40.804555,-73.939686,36,6.12,1,0,2
327,3509,40.801194,-73.950074,31,6.12,1,2,0


In [23]:
final_df = pd.concat([date_df, day_data], axis = 1)

In [24]:
dock_id_series = final_df['dock_id']

In [25]:
final_df.drop(['dock_id'], axis = 1, inplace = True)

In [26]:
final_df

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster
0,12,7,13,40.683826,-73.976323,62,6.12,1,2,3
1,12,7,13,40.696089,-73.978034,19,6.12,1,2,0
2,12,7,13,40.686768,-73.959282,19,6.12,1,1,0
3,12,7,13,40.692395,-73.993379,24,6.12,1,2,3
4,12,7,13,40.698399,-73.980689,19,6.12,1,2,0
...,...,...,...,...,...,...,...,...,...,...
324,12,7,13,40.805726,-73.936322,33,6.12,1,2,2
325,12,7,13,40.801307,-73.939817,29,6.12,1,0,2
326,12,7,13,40.804555,-73.939686,36,6.12,1,0,2
327,12,7,13,40.801194,-73.950074,31,6.12,1,2,0


In [27]:
predictions = model.best_estimator_.predict(final_df)

In [28]:
final_df['predictions'] = predictions
final_df['dock_id'] = dock_id_series


In [29]:
final_df['avail_bikes_proportion'] = final_df['predictions']/final_df['tot_docks']

In [30]:
final_df

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,avail_bikes_proportion
0,12,7,13,40.683826,-73.976323,62,6.12,1,2,3,28.140678,83,0.453882
1,12,7,13,40.696089,-73.978034,19,6.12,1,2,0,10.813116,119,0.569111
2,12,7,13,40.686768,-73.959282,19,6.12,1,1,0,6.326385,120,0.332968
3,12,7,13,40.692395,-73.993379,24,6.12,1,2,3,7.762112,143,0.323421
4,12,7,13,40.698399,-73.980689,19,6.12,1,2,0,11.109045,144,0.584687
...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,12,7,13,40.805726,-73.936322,33,6.12,1,2,2,12.372473,3505,0.374923
325,12,7,13,40.801307,-73.939817,29,6.12,1,0,2,3.261555,3506,0.112467
326,12,7,13,40.804555,-73.939686,36,6.12,1,0,2,25.702741,3507,0.713965
327,12,7,13,40.801194,-73.950074,31,6.12,1,2,0,22.540406,3509,0.727110


In [31]:
def manhattan_distance(start_lat, start_lon, end_lat, end_lon):
    dist = distance((start_lat, start_lon), (start_lat, end_lon)).miles + \
           distance((end_lat, end_lon), (start_lat, end_lon)).miles
    return dist

In [32]:
low_bike_threshold = 1/3
high_bike_threshold = 2/3
max_distance = 2
max_bikes_rebalanced = 200
min_cargo_size = 3

In [33]:
data_low = final_df[final_df['avail_bikes_proportion'] <= low_bike_threshold]

In [34]:
data_high = final_df[final_df['avail_bikes_proportion'] >= high_bike_threshold]

In [35]:
data_low['deficit'] = round((low_bike_threshold - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')

<ipython-input-35-d746bb208678>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_low['deficit'] = round((low_bike_threshold - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')


In [36]:
data_high['surplus'] = round((data_high['avail_bikes_proportion'] - high_bike_threshold) * data_high['tot_docks']).astype('int')

<ipython-input-36-d9c66eeb9a9d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_high['surplus'] = round((data_high['avail_bikes_proportion'] - high_bike_threshold) * data_high['tot_docks']).astype('int')


In [37]:
data_low = data_low.sort_values(by = 'deficit', ascending = False)

In [38]:
data_high = data_high.sort_values(by = 'surplus', ascending = False)

In [39]:
rebalancing_dict = {}

In [40]:
low_copy = data_low.copy()
high_copy = data_high.copy()

In [41]:
bikes_rebalanced = 0
for low in low_copy.index:
    if low_copy.loc[low, 'deficit'] == 0:
        continue
    for high in high_copy.index:
        if high_copy.loc[high, 'surplus'] == 0:
            continue
        if manhattan_distance(low_copy.loc[low, 'latitude'], low_copy.loc[low, 'longitude'], 
                              high_copy.loc[high, 'latitude'], high_copy.loc[high, 'longitude']) < max_distance:
            stations_key = (low_copy.loc[low, 'dock_id'], high_copy.loc[high, 'dock_id'])
            change = min(low_copy.loc[low, 'deficit'], high_copy.loc[high, 'surplus'])
            low_copy.loc[low, 'deficit'] -= change
            high_copy.loc[high, 'surplus'] -= change
            bikes_rebalanced += change
            if stations_key in rebalancing_dict.keys():
                rebalancing_dict[stations_key] += change
            else:
                rebalancing_dict[stations_key] = change
            if low_copy.loc[low, 'deficit'] == 0:
                break

In [42]:
rebalancing_dict

{(469, 3165): 7,
 (469, 3158): 5,
 (469, 3137): 4,
 (469, 3163): 1,
 (195, 355): 10,
 (195, 368): 3,
 (3437, 3374): 9,
 (3437, 3387): 3,
 (164, 3163): 3,
 (164, 3128): 1,
 (3134, 3356): 4,
 (3141, 3318): 7,
 (3141, 3312): 4,
 (3328, 3500): 6,
 (3328, 3312): 1,
 (3328, 3387): 2,
 (3328, 3296): 2,
 (3146, 3296): 2,
 (3284, 3509): 2,
 (3294, 3497): 3,
 (3320, 3507): 2,
 (3479, 3409): 7,
 (3479, 3342): 2,
 (3462, 334): 3,
 (528, 168): 9,
 (2012, 168): 3,
 (2012, 439): 6,
 (524, 462): 7,
 (524, 483): 1,
 (3127, 3123): 5,
 (3127, 3109): 3,
 (3418, 3324): 8,
 (3075, 539): 7,
 (224, 439): 3,
 (224, 368): 3,
 (448, 3224): 5,
 (491, 368): 2,
 (491, 3224): 3,
 (3122, 3109): 1,
 (3452, 539): 1,
 (3452, 2002): 3,
 (3358, 3416): 4,
 (167, 357): 1,
 (3384, 3416): 3,
 (3384, 3410): 1,
 (406, 3410): 4,
 (3310, 3410): 2,
 (3310, 3419): 2,
 (3306, 310): 3,
 (3449, 2002): 3,
 (245, 310): 3,
 (249, 3427): 3,
 (389, 296): 3,
 (2005, 3429): 2,
 (3050, 3429): 2,
 (3103, 3070): 2,
 (3073, 3070): 2,
 (3101, 306

In [43]:
sorted_rebalancing = dict(sorted(rebalancing_dict.items(), key=lambda x: x[1], reverse = True))

In [44]:
filtered_rebalancing = {key: value for key, value in sorted_rebalancing.items() if value >= min_cargo_size}

In [45]:
filtered_rebalancing

{(195, 355): 10,
 (3437, 3374): 9,
 (528, 168): 9,
 (3418, 3324): 8,
 (469, 3165): 7,
 (3141, 3318): 7,
 (3479, 3409): 7,
 (524, 462): 7,
 (3075, 539): 7,
 (3328, 3500): 6,
 (2012, 439): 6,
 (469, 3158): 5,
 (3127, 3123): 5,
 (448, 3224): 5,
 (469, 3137): 4,
 (3134, 3356): 4,
 (3141, 3312): 4,
 (3358, 3416): 4,
 (406, 3410): 4,
 (195, 368): 3,
 (3437, 3387): 3,
 (164, 3163): 3,
 (3294, 3497): 3,
 (3462, 334): 3,
 (2012, 168): 3,
 (3127, 3109): 3,
 (224, 439): 3,
 (224, 368): 3,
 (491, 3224): 3,
 (3452, 2002): 3,
 (3384, 3416): 3,
 (3306, 310): 3,
 (3449, 2002): 3,
 (245, 310): 3,
 (249, 3427): 3,
 (389, 296): 3}

In [46]:
filtered_bikes_rebalanced = 0
final_rebalancing_dict = {}
for k, v in filtered_rebalancing.items():
    if filtered_bikes_rebalanced < max_bikes_rebalanced:
        final_rebalancing_dict[k] = v
        filtered_bikes_rebalanced += v

In [47]:
rebalancing_df = pd.DataFrame(final_rebalancing_dict.items(), columns = ['dock_ids', 'num_bikes'])

In [48]:
rebalancing_df[['dock_id_receive', 'dock_id_give']] = rebalancing_df['dock_ids'].tolist()
rebalancing_df.drop(['dock_ids'], axis = 1, inplace = True)

In [49]:
data_df = final_df[['dock_id', 'latitude', 'longitude']]

In [50]:
new_docks = new[['dock_id', 'dock_name']]

In [51]:
rebalancing_df = rebalancing_df.merge(data_df, how = 'left', left_on = 'dock_id_receive', right_on = 'dock_id').rename(
    columns = {'latitude': 'latitude_receive', 'longitude': 'longitude_receive'})
rebalancing_df = rebalancing_df.merge(data_df, how = 'left', left_on = 'dock_id_give', right_on = 'dock_id').rename(
    columns = {'latitude': 'latitude_give', 'longitude': 'longitude_give'})
rebalancing_df.drop(['dock_id_x', 'dock_id_y'], axis = 1, inplace = True)

In [52]:
rebalancing_df = rebalancing_df.merge(new_docks, how = 'left', left_on = 'dock_id_receive', right_on = 'dock_id').rename(
    columns = {'dock_name': 'dock_name_receive'})
rebalancing_df = rebalancing_df.merge(new_docks, how = 'left', left_on = 'dock_id_give', right_on = 'dock_id').rename(
    columns = {'dock_name': 'dock_name_give'})
rebalancing_df.drop(['dock_id_x', 'dock_id_y'], axis = 1, inplace = True)

In [61]:
GREEN_RGB = [0, 255, 0, 150]
RED_RGB = [240, 100, 0, 150]

# Specify a deck.gl ArcLayer
arc_layer = pdk.Layer(
    "ArcLayer",
    data = rebalancing_df,
    get_width="num_bikes",
    get_source_position=["longitude_give", "latitude_give"],
    get_target_position=["longitude_receive", "latitude_receive"],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    pickable=True,
    auto_highlight=True,
)

view_state = pdk.ViewState(latitude=40.74, longitude=-74, bearing=45, pitch=50, zoom=8,)


TOOLTIP_TEXT = {"html": "{num_bikes} bikes need rebalancing from<br />{dock_name_give} to {dock_name_receive}"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT, map_style = 'light')
r.to_html("saturday_1pm_rebalancing.html")
with open('saturday_1pm_rebalancing.json', 'w') as outfile:
    json.dump(r.to_json(), outfile)

In [54]:
bikes_rebalanced

207

In [55]:
filtered_bikes_rebalanced

169

In [56]:
data_low['deficit'].sum()

627

In [57]:
data_high['surplus'].sum()

378

In [58]:
data_low.head(5)

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,avail_bikes_proportion,deficit
79,12,7,13,40.763441,-73.982681,57,6.12,1,0,1,2.472330,469,0.043374,17
10,12,7,13,40.709056,-74.010434,45,6.12,1,5,1,1.930735,195,0.042905,13
287,12,7,13,40.793135,-73.977004,39,6.12,1,1,2,1.197220,3437,0.030698,12
6,12,7,13,40.753231,-73.970325,47,6.12,1,5,1,3.566825,164,0.075890,12
138,12,7,13,40.763126,-73.965269,51,6.12,1,0,1,4.641171,3134,0.091003,12


In [59]:
data_high.head(5)

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,avail_bikes_proportion,surplus
8,12,7,13,40.739713,-73.994564,47,6.12,1,5,3,43.146178,168,0.918004,12
48,12,7,13,40.716021,-73.999744,43,6.12,1,2,3,38.273538,355,0.890082,10
244,12,7,13,40.799484,-73.955613,36,6.12,1,3,3,33.240766,3374,0.923355,9
73,12,7,13,40.726281,-73.989780,39,6.12,1,4,3,34.626607,439,0.887862,9
204,12,7,13,40.668546,-73.993333,25,6.12,1,3,3,24.698553,3324,0.987942,8
